In [ ]:
####################################################################
# Prepared for Gabor's Data Analysis
#
# Data Analysis for Business, Economics, and Policy
# by Gabor Bekes and  Gabor Kezdi
# Cambridge University Press 2021
#
# gabors-data-analysis.com
#
# License: Free to share, modify and use for educational purposes.
# 	Not to be used for commercial purposes.
#
####################################################################

####################################################################
# football dataset
#
# input:
#       fdata_pl_t.csv, t=2008-2018
#       epl_2007.csv
#       managers_epl.xlsx

# output:
#       epl-totalpoints.csv , t=2007-2018
#       epl-games.csv , t=2008-2018
#       epl-teams-games.csv , t=2008-2018
#       football-managers.csv , t=2008-2018
# workfile
#       football-managers-workfile.csv

# version 1.0   2021-08-27
####################################################################

In [ ]:
### SETTING UP DIRECTORIES

# import packages
import pandas as pd
import os
import numpy as np
import datetime as dt

# set working directory for da_data_repo -- replace the
os.chdir('YOUR PATH')

# location folders
data_in = "./football/raw/"
data_out = "./football/clean/"

In [ ]:
###list of games
#load csvs
dataframes_to_concat = []
for year in range(2008, 2019):
    df = pd.read_csv(data_in + f"fdata_pl_{year}.csv")
    dataframes_to_concat.append(df)
df = pd.concat(dataframes_to_concat, axis=0, ignore_index=True)

#select columns we will use
df.columns = df.columns.str.lower()
df = df.loc[df["hometeam"].notnull()]
df = df[["div", "date", "hometeam", "awayteam", "fthg", "ftag"]]

#gen id
df = df.sort_values(by=["hometeam"]).reset_index(drop=True)
df["hometeam_uid"] = df.groupby("hometeam").ngroup() + 1
df = df.sort_values(by=["awayteam"]).reset_index(drop=True)
df["awayteam_uid"] = df.groupby("awayteam").ngroup() + 1
df = df.loc[df["date"].notnull()]
df.rename(
    columns={
        "fthg": "goals_home",
        "ftag": "goals_away",
        "hometeam": "team_home",
        "awayteam": "team_away",
    },
    inplace=True,
)

#set date and gen season variable
df['date'] = pd.to_datetime(df['date'],dayfirst=True)
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df.loc[df['month'] <= 6, 'season'] = df.year-1
df.loc[df['month'] >=8, 'season'] = df.year
df['season']=df['season'].astype(int)
df = df.sort_values(['season', 'date']).reset_index(drop=True)

#calculate points
df.loc[df['goals_home'] > df['goals_away'], 'points_home'] = 3
df.loc[df['goals_home'] == df['goals_away'], 'points_home'] = 1
df.loc[df['goals_home'] < df['goals_away'], 'points_home'] = 0
df.loc[df['goals_home'] < df['goals_away'], 'points_away'] = 3
df.loc[df['goals_home'] == df['goals_away'], 'points_away'] = 1
df.loc[df['goals_home'] > df['goals_away'], 'points_away'] = 0

#arrange and save dataset
df=df[['div','season','date','team_home','team_away','points_home','points_away','goals_home','goals_away','hometeam_uid','awayteam_uid']].reset_index(drop=True)
df.to_csv(data_out + "epl-games_p.csv", index=False)

In [ ]:
#register games from both side
df = pd.read_csv(data_out + "epl-games_p.csv")
new_cols=[]
for col in df.columns:
    new_cols.append(col.replace('_home','').replace('_away','_opponent'))
df.columns=new_cols
df['home']=1
df_a = pd.read_csv(data_out + "epl-games_p.csv")
new_cols=[]
for col in df_a.columns:
    new_cols.append(col.replace('_away','').replace('_home','_opponent'))
df_a.columns=new_cols
df_a['home']=0
df=pd.concat([df,df_a])
del df_a

#gameno: game number team played in season
df = df.sort_values(['team','season', 'date']).reset_index(drop=True)
df['gameno']=df.groupby(['team','season']).cumcount()+1

#arrange and save dataset
first_cols = ['div', 'season', 'date', 'team', 'gameno', 'home', 'points', 'goals']
last_cols = [col for col in df.columns if col not in first_cols]
df = df[first_cols + last_cols]
df.to_csv(data_out + "epl-teams-games_p.csv", index=False)

In [ ]:
###calculate points of each team
#load missing data and calculate totalpoints
df_2007=pd.read_excel(data_in+'epl_2007.xlsx', index_col=False,sheet_name='Sheet1')
df_2007.columns=['team','goaldiff_season','points_season','season']

df = pd.read_csv(data_out + "epl-teams-games_p.csv")
df['goaldiff']=df.goals-df.goals_opponent
df = (
    df.groupby(["season", "team"])
    .agg(
        {
            "points": "sum",
            "goaldiff": "sum",
        }
    )
    .reset_index()
    .rename(columns={'points':'points_season','goaldiff' : 'goaldiff_season'}))

df=pd.concat([df,df_2007])
df = df.sort_values(['team', 'season']).reset_index(drop=True)
df.to_csv(data_out + "epl-totalpoints_p.csv", index=False)

In [ ]:
###list of managers
df=pd.read_excel(data_in+'managers_epl.xlsx', index_col=False,sheet_name='epl')
df.columns=df.columns.str.lower()

df['caretaker'] = df['name'].str.contains('double-dagger').astype(int)
df['name']=df['name'].str.replace("double-dagger","").str.replace("dagger","").str.replace("§","").str.strip()

df['until']=df['until'].astype(str)
df['from']=df['from'].astype(str)
df['until']=df['until'].str.split('[').str[0]
df['from']=df['from'].str.split('[').str[0]
df['flag_inoffice'] = (df['until']=="Present*").astype(int)
df["until"].replace({"Present*": "1 July 2019"}, inplace=True)
df["date_until"]=pd.to_datetime(df['until'],dayfirst=True)
df["date_from"]=pd.to_datetime(df['from'],dayfirst=True)

df=df.drop_duplicates(subset=['name', 'club', 'date_from', 'date_until'], keep='first')

df=df.sort_values(by=['club','date_from']).reset_index(drop=True)
df['manager_id']=df.groupby('name').ngroup()+1
df['manager_name']=df['name']

df=df[['manager_id','manager_name','club','caretaker','date_until',
       'date_from','nat.','league', 'flag_inoffice' ]]
df.to_csv(data_out + "football-managers_p.csv", index=False)

In [ ]:
###find out which manager was being charge during the event
df = pd.read_csv(data_out + "football-managers_p.csv")

#use first word
team_label={x:x.split(' ')[0] for x in df['club'].unique()}
#exceptions
team_label["Aston Villa"]="Aston Villa"
team_label["Crystal Palace"]="Crystal Palace"
team_label["Manchester United"]="Man United"
team_label["Manchester City"]="Man City"
team_label["West Bromwich Albion"]="West Brom"
team_label["West Ham United"]="West Ham"
team_label["Queens Park Rangers"]="QPR"
team_label["Wolverhampton Wanderers"]="Wolves"
#map
df['team']=df['club'].map(team_label)

df["date_until"]=pd.to_datetime(df['date_until'])
df["date_from"]=pd.to_datetime(df['date_from'])
df=df.loc[df['date_until'].dt.year>=2008]

df=df[['team', 'manager_id', 'manager_name','date_from','date_until', 'caretaker']]
df=df.sort_values(by=['team','date_from']).reset_index(drop=True)

df['manager_no']=df.groupby(['team']).cumcount()+1

piv = df.pivot(
            index=["team"],
            columns=["manager_no"],
            values=["manager_id", "manager_name", "date_from","date_until","caretaker"],
        )

piv.columns=[('{0}{1}'.format(*tup)) for tup in piv.columns]
piv=piv.reset_index()

df_left=pd.read_csv(data_out + "epl-teams-games_p.csv")
df = pd.merge(df_left, piv, on='team',how='left')

df["date"]=pd.to_datetime(df['date'])

#iter through the dataframe and find the matching time intervall
manager_names,manager_ids,caretakers,date_froms,date_untils=[],[],[],[],[]
for x in df.iterrows():
    row=x[1]
    date=row['date']
    for i in range(1,14):
        manager_name=''
        manager_id=''
        caretaker=''
        date_from=''
        date_until=''
        if (date>=row[f'date_from{i}'])&(date<=row[f'date_until{i}']): #the column number will define the manager
            manager_name=row[f'manager_name{i}']
            manager_id=row[f'manager_id{i}']
            caretaker=row[f'caretaker{i}']
            date_from=row[f'date_from{i}']
            date_until=row[f'date_until{i}']
            break      
    manager_names.append(manager_name)
    manager_ids.append(manager_id)
    caretakers.append(caretaker)
    date_froms.append(date_from)
    date_untils.append(date_until)
            
df['manager_name']=manager_names
df['manager_id']=manager_ids
df['caretaker']=caretakers
df['date_from']=date_froms
df['date_until']=date_untils

df=df[['div', 'season', 'date', 'team', 'gameno', 'home', 'points', 'goals',
       'team_opponent', 'points_opponent', 'goals_opponent', 'hometeam_uid',
       'awayteam_uid', 'manager_name', 'manager_id',
       'caretaker','date_from', 'date_until']]

#missing data filled by hand
df.loc[(df.team=='Reading')&(df.gameno==30),['manager_name','manager_id','caretaker','date_from','date_until']]=['Eanonn Dolan',262,1,dt.datetime(2013,3,11),dt.datetime(2013,3,26)]

#points of last season
df_l = pd.read_csv(data_out + "epl-totalpoints_p.csv")

df_l.loc[df_l['team']==df_l['team'].shift(1),'points_lastseason']=df_l['points_season'].shift(1)

df_l=df_l[['team', 'season', 'points_lastseason']]

df = pd.merge(df, df_l, on=['season','team'],how='left')

df.to_csv(data_out + "football-managers-workfile_p.csv", index=False)